In [11]:
import zipfile
output ="/home/rajat/Downloads/dataset_1.zip"
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [1]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder):
    images = []
    labels = []
    for subdir, _, files in os.walk(folder):
        for file in files:
            img_path = os.path.join(subdir, file)
            img = cv2.imread(img_path)
            if img is not None:
                
                img = cv2.resize(img, (128, 128))
                
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                gray = gray / 255.0
                images.append(gray)
                
                label = os.path.basename(subdir)
                labels.append(label)
    return np.array(images), np.array(labels)

images, labels = load_images_from_folder('dataset')


In [3]:
from skimage.feature import hog
from skimage.feature import local_binary_pattern

def extract_hog(image):
    if image is None or len(image.shape) != 2:
        return np.array([])
    hog_features, hog_image = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return hog_features


def build_filters():
    filters = []
    ksize = 31
    for theta in np.arange(0, np.pi, np.pi / 4):
        for sigma in (1, 3):
            for lambd in np.arange(0, np.pi, np.pi / 4):
                kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, 0.5, 0, ktype=cv2.CV_32F)
                filters.append(kernel)
    return filters

def process(image, filters):
    if image is None or len(image.shape) != 2:
        return np.array([])
    accum = np.zeros_like(image)
    for kernel in filters:
        fimg = cv2.filter2D(image, cv2.CV_8UC3, kernel)
        np.maximum(accum, fimg, accum)
    return accum.flatten()

filters = build_filters()

def extract_lbp(image, P=8, R=1):
    if image is None or len(image.shape) != 2:
        return np.array([])
    lbp = local_binary_pattern(image, P, R, method='uniform')
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp, bins=n_bins, range=(0, n_bins))
    return hist

def extract_sift(image):
    if image is None or len(image.shape) != 2 or image.dtype != np.uint8:
        return np.array([])
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is not None:
        return descriptors.flatten()
    else:
        return np.array([])
    
def extract_combined_features(image):
    if image is None or len(image.shape) != 2:
        return np.array([])

    hog_features = extract_hog(image)
    gabor_features = process(image, filters)
    lbp_features = extract_lbp(image)
    sift_features = extract_sift(image)

    # Combine features ensuring they are of the same size
    combined_features = np.hstack((hog_features, gabor_features, lbp_features, sift_features))

    # Handling cases where some features might be empty by padding with zeros
    expected_length = 128 * 128 * 4  # Example expected length, adjust as needed
    if combined_features.size < expected_length:
        combined_features = np.pad(combined_features, (0, expected_length - combined_features.size), 'constant')

    return combined_features

combined_features = np.array([extract_combined_features(img) for img in images])


/home/rajat/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
from sklearn.decomposition import PCA

# Apply PCA for Dimensionality Reduction
pca = PCA(n_components=6)  # Adjust the number of components as needed
combined_features_reduced = pca.fit_transform(combined_features)

# Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.2, random_state=42)

# Train a Classifier (e.g., SVM)
# from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# clf = SVC(kernel='rbf')
clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(X_train, y_train)

# Evaluate the Model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.7550047664442326
Classification Report:
               precision    recall  f1-score   support

    Building       0.68      0.51      0.58        92
      Forest       0.83      0.99      0.90       526
     Glacier       0.65      0.55      0.59       106
   Mountains       0.60      0.60      0.60       112
         Sea       0.62      0.54      0.57       108
     Streets       0.77      0.41      0.53       105

    accuracy                           0.76      1049
   macro avg       0.69      0.60      0.63      1049
weighted avg       0.74      0.76      0.74      1049

Confusion Matrix:
 [[ 47  24   3   5   2  11]
 [  0 519   0   4   2   1]
 [  1  20  58  13  13   1]
 [  2  14  11  67  18   0]
 [  3  13  12  22  58   0]
 [ 16  39   5   1   1  43]]


In [10]:
import joblib

joblib.dump(clf, "BombaymodelRF.pkl")
# joblib.dump(pca, "Bombaypca.pkl")

['BombaymodelRF.pkl']

In [11]:
from flask import Flask, request, jsonify, render_template
import cv2
import numpy as np
import joblib

app = Flask(__name__)

# Load the trained model and PCA
model = joblib.load('BombaymodelRF.pkl')
# pca = joblib.load('Bombaypca.pkl')

# Load the filters for Gabor
filters = build_filters()

def preprocess_image(image):
    # Resize, grayscale, normalize, and extract features
    if image is None or image.size == 0:
        return None
    image = cv2.resize(image, (128, 128))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = gray / 255.0
    features = extract_combined_features(gray)
    # features_reduced = pca.transform([features])
    return features.reshape(1,-1)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return "No file part"
    file = request.files['file']
    if file.filename == '':
        return "No selected file"
    if file:
        image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
        features = preprocess_image(image)
        if features is None or features.shape[0] == 0:
            return jsonify({'error': 'Invalid image'})
        prediction = model.predict(features)
        return jsonify({'prediction': prediction[0]})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Jun/2024 12:50:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2024 12:50:08] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2024 12:50:53] "POST /predict HTTP/1.1" 200 -
